In [8]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [9]:
# Colab 폴더로 경로 이동
%cd /content/drive/MyDrive/Colab Notebooks/DeepLearning

/content/drive/MyDrive/Colab Notebooks/DeepLearning


# '3D-mnist'
---
### [1] 데이터 준비



In [10]:
import numpy as np
import pandas as pd

from keras.layers import Conv3D, MaxPool3D, Flatten, Dense
from keras.layers import Dropout, Input, BatchNormalization, Activation
from sklearn.metrics import confusion_matrix, accuracy_score
from keras.losses import categorical_crossentropy
from keras.models import Model
from keras.utils.np_utils import to_categorical

import matplotlib.pyplot as plt
from matplotlib.pyplot import cm
%matplotlib inline

import seaborn as sns

import h5py

In [11]:
with h5py.File('../Kaggle/3D_MNIST/full_dataset_vectors.h5', "r") as hf:    
    # Split the data into training/test features/targets
    x_train = hf["X_train"][:]
    y_train = hf["y_train"][:]
    x_test = hf["X_test"][:] 
    y_test = hf["y_test"][:]

print(f'x_train_shape: {x_train.shape}, y_train_shape: {y_train.shape}')
print(f'x_test_shape : {x_test.shape},  y_test_shape : {y_test.shape}')

x_train_shape: (10000, 4096), y_train_shape: (10000,)
x_test_shape : (2000, 4096),  y_test_shape : (2000,)


In [12]:
y_train[1], y_test[1]

(5, 7)

In [13]:
## Introduce the channel dimention in the input dataset 
xtrain = np.ndarray((x_train.shape[0], 4096, 3))
xtest = np.ndarray((x_test.shape[0], 4096, 3))

# Translate data to color
def add_rgb_dimention(array):
    scalar_map = cm.ScalarMappable(cmap="Oranges")
    return scalar_map.to_rgba(array)[:, : -1]

## iterate in train and test, add the rgb dimention 
for i in range(x_train.shape[0]):
    xtrain[i] = add_rgb_dimention(x_train[i])
for i in range(x_test.shape[0]):
    xtest[i] = add_rgb_dimention(x_test[i])

## convert to 1 + 4D space (1st argument represents number of rows in the dataset)
xtrain = xtrain.reshape(x_train.shape[0], 16, 16, 16, 3)
xtest = xtest.reshape(x_test.shape[0], 16, 16, 16, 3)

## convert target variable into one-hot
y_train = to_categorical(y_train, 10)

In [14]:
xtrain.shape, y_train.shape

((10000, 16, 16, 16, 3), (10000, 10))

In [15]:
## input layer
input_layer = Input((16, 16, 16, 3))

## convolutional layers
x = Conv3D(filters=8, kernel_size=(3, 3, 3), use_bias=False, padding='Same')(input_layer)
x = BatchNormalization()(x)
x = Activation("relu")(x)
x = Conv3D(filters=16, kernel_size=(3, 3, 3), use_bias=False, padding='Same')(x)
x = BatchNormalization()(x)
x = Activation("relu")(x)

## Pooling layer
x = MaxPool3D(pool_size=(2, 2, 2))(x) # the pool_size (2, 2, 2) halves the size of its input

## convolutional layers
x = Conv3D(filters=32, kernel_size=(3, 3, 3), use_bias=False, padding='Same')(x)
x = BatchNormalization()(x)
x = Activation("relu")(x)
x = Conv3D(filters=64, kernel_size=(3, 3, 3), use_bias=False, padding='Same')(x)
x = BatchNormalization()(x)
x = Activation("relu")(x)

## Pooling layer
x = MaxPool3D(pool_size=(2, 2, 2))(x)
x = Dropout(0.25)(x) #No more BatchNorm after this layer because we introduce Dropout

x = Flatten()(x)

## Dense layers
x = Dense(units=4096, activation='relu')(x)
x = Dropout(0.5)(x)
x = Dense(units=1024, activation='relu')(x)
x = Dropout(0.5)(x)
output_layer = Dense(units=10, activation='softmax')(x)

## define the model with input layer and output layer
model = Model(inputs=input_layer, outputs=output_layer, name="3D-CNN")
model_name = model.name

#It provides an optimization algorithm that can handle sparse gradients on noisy problems. The default configuration parameters do well on most problems.""
model.compile(loss='categorical_crossentropy', 
              optimizer=adam(), #default: lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0
              metrics=['acc'])

model.summary()

NameError: ignored